In [79]:
%run SharedParams.ipynb
%run MerkleTree.ipynb
fib_a = F_gen

fiat_shamir = FiatShamir()
proof_data = []

In [80]:
fib_points = []
c = F_gen
d = F_gen
for i in range(last_fib_index + 1):
    fib_points.append((g**(RS_factor*i), c))
    c, d = d, c+d
trace_poly = PR.lagrange_polynomial(fib_points)

In [81]:
coset_eval = []
for i in range(g_order):
    coset_eval.append(trace_poly(w * g**i))


In [82]:
fibonacci_constraint_trace_poly = trace_poly(PR(x)) + trace_poly(gamma * PR(x)) - trace_poly(PR(x) * gamma ** 2)
fibonacci_constraint_missing_zeroes_poly = 1
for i in range(last_fib_index - 1, trace_size):
    fibonacci_constraint_missing_zeroes_poly *= (PR(x) - gamma**i)
# For optimization - change PR(x)**trace_size to a faster implementation
fibonacci_constraint_zeroes_poly = (PR(x)**trace_size - PR(1)) / fibonacci_constraint_missing_zeroes_poly
ld_fibonacci_constraint_trace_poly = PR(fibonacci_constraint_trace_poly / fibonacci_constraint_zeroes_poly)

In [83]:
boundary_constraint_1 = PR((trace_poly - fib_1)/(PR(x)-1))
boundary_constraint_last = PR((trace_poly - fib_last)/(PR(x) - gamma**last_fib_index))

In [84]:
eval_merkle = MerkleTree(coset_eval)
proof_data.append(f"eval_merkle:{eval_merkle.root}")
beta0, beta1, beta2, beta3, beta4, beta5 = (fiat_shamir.get_random_int(proof_data) % P for i in range(6))

cp = beta0 + beta1 * (ld_fibonacci_constraint_trace_poly * PR(x) ** (1023 - ld_fibonacci_constraint_trace_poly.degree()))
cp += beta2 + beta3 * (boundary_constraint_1 * PR(x) ** (1023 - boundary_constraint_1.degree()))
cp += beta4 + beta5 * (boundary_constraint_last * PR(x) ** (1023 - boundary_constraint_last.degree()))



In [85]:
cp_eval = []
for i in range(g_order):
    cp_eval.append(cp(w * g**i))